# 使用ONNX将模型转移至Caffe2和移动端

在[本教程](https://pytorch123.com/FourSection/ONNX/)中，我们将介绍如何使用 ONNX 将 PyTorch 中定义的模型转换为 ONNX 格式，然后将其加载到 Caffe2 中。一旦进入 Caffe2，我们 就可以运行模型来仔细检查它是否正确导出，然后我们展示了如何使用 Caffe2 功能（如移动导出器）在移动设备上执行模型。

在本教程中，您需要安装onnx和Caffe2。 您可以使用pip install onnx来获取 onnx。

注意：本教程需要 PyTorch master 分支，可以按照这里说明进行安装。

## 1 引入模型

In [6]:
# 一些包的导入
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

### 1.1 SuperResolution Model
超分辨率是一种提高图像、视频分辨率的方法，广泛用于图像处理或视频剪辑。在本教程中，我们将首先使用带有虚拟输入的小型超分辨率模型。

首先，让我们在 PyTorch 中创建一个SuperResolution模型。这个模型 直接来自 PyTorch 的例子，没有修改：

In [7]:
# PyTorch中定义的Super Resolution模型
import torch.nn as nn
import torch.nn.init as init

class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# 使用上面模型定义，创建super-resolution模型 
torch_model = SuperResolutionNet(upscale_factor=3)

### 1.2 Train Model

通常，你现在会训练这个模型; 但是，对于本教程我们将下载一些预先训练的权重。请注意，此模型未经过充分训练来获得良好的准确性，此处 仅用于演示目的。

In [8]:
# 加载预先训练好的模型权重
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1    # just a random number

# 使用预训练的权重初始化模型
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))

# 将训练模式设置为falsesince we will only run the forward pass.
torch_model.train(False)

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)

### 1.3 导出模型
在 PyTorch 中通过跟踪工作导出模型。要导出模型，请调用torch.onnx._export()函数。这将执行模型，记录运算符用于计算输出的轨迹。 因为_export运行模型，我们需要提供输入张量x。这个张量的值并不重要; 它可以是图像或随机张量，只要它大小是正确的。

要了解有关PyTorch导出界面的更多详细信息，请查看[torch.onnx documentation](https://pytorch.org/docs/master/onnx.html)文档。

In [9]:
# 输入模型
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)

# 导出模型
torch_out = torch.onnx._export(torch_model,             # model being run
                               x,                       # model input (or a tuple for multiple inputs)
                               "../../data/super_resolution.onnx", # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file


/tmp/ipykernel_703365/2513470022.py:5: FutureWarning: 'torch.onnx._export' is deprecated in version 1.12.0 and will be removed in 2.0. Please use `torch.onnx.export` instead.
  torch_out = torch.onnx._export(torch_model,             # model being run


torch_out是执行模型后的输出。通常您可以忽略此输出，但在这里我们将使用它来验证我们导出的模型在Caffe2中运行时是否计算出相同的值。

### 1.4 采用ONNX表示模型并在Caffe2中使用
现在让我们采用 ONNX 表示并在 Caffe2 中使用它。这部分通常可以在一个单独的进程中或在另一台机器上完成，但我们将在同一个进程中继续， 以便我们可以验证 Caffe2 和 PyTorch 是否为网络计算出相同的值：

In [5]:
import onnx
import caffe2.python.onnx.backend as onnx_caffe2_backend

#加载ONNX ModelProto对象。模型是一个标准的Python protobuf对象
model = onnx.load("super_resolution.onnx")

# 为执行模型准备caffe2后端，将ONNX模型转换为可以执行它的Caffe2 NetDef。 
# 其他ONNX后端，如CNTK的后端即将推出。
prepared_backend = onnx_caffe2_backend.prepare(model)

# 在Caffe2中运行模型

# 构造从输入名称到Tensor数据的映射。
# 模型图形本身包含输入图像之后所有权重参数的输入。由于权重已经嵌入，我们只需要传递输入图像。 
# 设置第一个输入。
W = {model.graph.input[0].name: x.data.numpy()}

# 运行Caffe2 net:
c2_out = prepared_backend.run(W)[0]

# 验证数字正确性，最多3位小数
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), c2_out, decimal=3)

print("Exported model has been executed on Caffe2 backend, and the result looks good!")

ModuleNotFoundError: No module named 'caffe2'